[A full list of possible constraints and preferences is available in the reference manual.](https://docs.classiq.io/latest/reference-manual/platform/synthesis/constraints/)

# Concrete Example
First, understand Qmod through an example.

The task is to design a quantum algorithm that coherently computes the arithmetic operation $y=x^2+1$, for a quantum variable $\Ket{x}$ taht is a superposition of all the numbers between 0 and 7: $\Ket{x} = \frac{1}{\sqrt{8}}(\Ket{0}+\Ket{1}+ ... + \Ket{7})$. The expected output is
$$\Ket{x}\Ket{y} = \Ket{x}\Ket{x^2+1} = \frac{1}{\sqrt{8}}\sum^{7}_{i=0}\Ket{i}\Ket{i^2+1}$$, where $\Ket{x}$ is entangled to $\Ket{y}$.
This may sound complicated, but the following few lines of code in Qmod create the desired algorithm with Classiq.

In [1]:
from classiq import *


@qfunc
def main(x: Output[QNum], y: Output[QNum]):

    allocate(4, x)
    hadamard_transform(x)  # creates a uniform superposition
    y |= x**2 + 1

Which one can always directly synthesize without any constraints or preferences:

In [9]:
quantum_program = synthesize(create_model(main))

But in the following we cover how to do apply constraints and preferences.

### Constraints and Preferences in the Python SDK
The synthesis engine receives an underlying Qmod representation of the quantum model that is constructed in the Python SDK using the create_model function:

In [2]:
quantum_model = create_model(main)

This quantum_model can be directly synthesized with the command synthesize(quantum_model) to return the quantum program implementation. However, if we want to apply some constraints and preferences we need to adapt the quantum_model representation.

First, assume that we want to receive a circuit implementation with minimal number of qubits and with maximal circuit depth of 500
. So we can apply these as constraints to our quantum_model:

In [11]:
quantum_model_with_constraints = set_constraints(
    quantum_model, Constraints(optimization_parameter="width", max_depth=500)
)
quantum_program = synthesize(quantum_model_with_constraints)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/578a595b-cebe-4eee-8b8a-e239fc7403d2?version=0.42.2


And the parameters of the circuit implementation can be extracted:

In [7]:
circuit_width = QuantumProgram.from_qprog(quantum_program).data.width
circuit_depth = QuantumProgram.from_qprog(quantum_program).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 12 and the circuit_depth is 392


**Compilation vs Transpilation**

The synthesis engine is a compiler that compiles a high-level functional model to one specific circuit out of many possible implementations. A transpiler on the other hand, transforms one circuit implementation to another one. Its use can be to change from a circuit representation with a given basis gate set to another one, or to further optimize a given circuit implementation with basic optimization procedures like cancellation of two identical Hermitian gates applied consequently.

Modify the constraints above such that you optimize the circuit for minimal circuit depth using maximum 25 qubits. What are the circuit depth and width you receive? Are they different than the above example? Try to analyze the two quantum circuits using `show(quantum_program)` and figure our which functional building block is implemented differently.

In [15]:
quantum_model_with_constraints2 = set_constraints(
    quantum_model, Constraints(optimization_parameter =
                               OptimizationParameter.DEPTH,max_width = 25)
)
quantum_program2 = synthesize(quantum_model_with_constraints2)
show(quantum_program2)

Opening: https://platform.classiq.io/circuit/75879705-3b06-4c55-a18e-38ac451d745c?version=0.42.2


In [16]:
circuit_width = QuantumProgram.from_qprog(quantum_program2).data.width
circuit_depth = QuantumProgram.from_qprog(quantum_program2).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 21 and the circuit_depth is 346


In [17]:
quantum_model_with_preferences = set_preferences(
    quantum_model,
    Preferences(backend_service_provider="IBM Quantum", backend_name="ibm_brisbane"),
)

quantum_program3 = synthesize(quantum_model_with_preferences)
circuit_width = QuantumProgram.from_qprog(quantum_program3).data.width
circuit_depth = QuantumProgram.from_qprog(quantum_program3).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 12 and the circuit_depth is 1655


In [18]:
write_qmod(quantum_model_with_constraints, "optimize")